### Tutorial - Introduction to scRNA-seq analysis using Scanpy
This Tutorial was guided heavily by Malte Luecken's "Best Practices" Notebook. (PostDoc from TheisLab) <br>
The original notebook can be found at https://github.com/theislab/single-cell-tutorial/blob/master/latest_notebook/Case-study_Mouse-intestinal-epithelium_1906.ipynb

It is highly recommended to look at the Scanpy Documentation to see a more detailed explanation on the function and other parameters one can specifiy: <br>
https://scanpy.readthedocs.io/en/stable/  

### Webtool to Longitudinal single cell transcriptomics:
https://theislab.github.io/LungInjuryRegeneration/
### Webtool to IPF Cell Atlas:
http://www.ipfcellatlas.com/

In [ ]:
## Import Libraries used in this script
import numpy as np
import matplotlib.pyplot as plt
import scanpy as sc
import pandas as pd
import seaborn as sb

import warnings
warnings.filterwarnings("ignore")

## Define a nice colour map for gene expression
from matplotlib import colors
gray_red = colors.LinearSegmentedColormap.from_list("", ["lightgray", "red", "darkred"], N = 128)

sc.settings.figdir = "Plots/"
sc.set_figure_params(vector_friendly = True)
plt.rcParams.update({"font.size": 14})
sb.set_style("ticks")

sc.logging.print_version_and_date()

In [ ]:
adata = sc.read("Data/Aging_Tutorial.h5ad")

In [ ]:
adata

In [ ]:
adata.obs.head()

In [ ]:
adata.obs.cell_type.value_counts()

In [ ]:
## See the first 10 gene names
print(adata.var_names[0:10])

## Show the first 10 cell ids
print(adata.obs_names[0:10])

## Show the expression values of the first 10 genes in the first 5 cells
## adata objects store the expression matrix in .X layer (cells (rows) x genes (columns))
adata.X[0:5, 0:10]

In [ ]:
## Show cell type numbers split by age
pd.crosstab(adata.obs.age, adata.obs.cell_type)

### Plot QC Metrics

In [ ]:
plt.rcParams['figure.figsize'] = (8, 6)
sc.pl.violin(adata, "n_counts", groupby = "identifier", size = 1.5, rotation = 90)
sc.pl.violin(adata, "percent_mito", groupby = "identifier", size = 1.5, rotation = 90)

In [ ]:
#Data quality summary plots
plt.rcParams['figure.figsize'] = (6, 5)

sc.pl.scatter(adata, "n_counts", "n_genes", color = "identifier", size = 8)

## Zoom in to range
sc.pl.scatter(adata[adata.obs["n_counts"] < 8000],"n_counts","n_genes", color = "percent_mito", size = 8)

In [ ]:
fig, axs = plt.subplots(nrows = 1, ncols = 2, figsize = (12, 5))
axs = axs.ravel()
sb.distplot(adata.obs["n_genes"], kde = False, ax = axs[0])
sb.distplot(adata.obs["percent_mito"], kde = False, ax = axs[1])
plt.show()

#### Raw counts are stored in adata.layers["counts"]

In [ ]:
print("raw counts\n",  adata.layers["counts"][0:5, 100:110].todense())

#### Normalized and log transformed counts are stored in adata.X 
These values are used in all of the visualisazions later on, e.g. UMAP, dotplot... 

In [ ]:
print("processed Counts in X\n", adata.X[0:5, 100:110].todense())

### In this object, the Principal components, k-nearest neighbor graph and UMAP are already calculated
See entries "X_pca" and "X_umap" in obsm

In [ ]:
adata

### UMAP as lower-dimensional representation of the data set
Captures the global structure of the high dimensional data set, each dot represents a single cell which can be coloured by any annotation you have stored in adata.obs

In [ ]:
sc.pl.umap(adata, color = ["age", "identifier", "cell_type"], wspace = 0.25)

#### You can also overlay the labels onto the embedding

In [ ]:
sc.pl.umap(adata, color = ["leiden_1"], legend_loc = "on data", legend_fontsize = 15)

In [ ]:
sc.pl.umap(adata, color = ["cell_type"], legend_loc = "on data", legend_fontsize = 10)

### Save Plots
You can save the plot by specifying the save Parameter

In [ ]:
sc.pl.umap(adata, color = ["cell_type"], legend_loc = "on data", legend_fontsize = 10, save = "_celltype.pdf")

In [ ]:
sc.pl.umap(adata, color = ["n_counts", "n_genes", "percent_mito"], size = 20, cmap = "viridis")

#### and show gene expression Feature plots by giving gene names in the color paramter

In [ ]:
sc.pl.umap(adata, color = ["Epcam", "Ptprc", "Col1a1", "Cldn5"], cmap = gray_red, size = 30)

#### Highlight certain labels by specifying them with groups

In [ ]:
sc.pl.umap(adata, color = ["age"], groups = ["3m"], size = 20)

#### and show only cells from young mice, and colour them by cell type or expression

In [ ]:
cells = adata.obs.age == "3m"
sc.pl.umap(adata[cells], color = ["Sftpc", "Ear2", "cell_type"], cmap = gray_red, size = 20)

### Differential Gene Expression - All Markers Table

In [ ]:
## All Markers
sc.tl.rank_genes_groups(adata, groupby = 'cell_type', use_raw = False, n_genes = 500,
                        method = "wilcoxon", pts = True)

In [ ]:
## Combine into one Data Frame (comparable to Marker Table)
result = adata.uns['rank_genes_groups']
allMarkers = []
for cluster in result['names'].dtype.names:
    current = pd.DataFrame({"gene": result["names"][cluster], "score": result["scores"][cluster],
                            "logfoldchange": result["logfoldchanges"][cluster], "pval": result["pvals"][cluster],
                            "pval_adj": result["pvals_adj"][cluster], 
                            "pct_within": result["pts"].loc[result["names"][cluster]][cluster],
                            "pct_outside": result["pts_rest"].loc[result["names"][cluster]][cluster],
                            "cluster": cluster})
    allMarkers.append(current)
allMarkers = pd.concat(allMarkers)
allMarkers.head()

In [ ]:
## Write to file
allMarkers.to_csv("Data/AllMarkers_cell_type.txt", sep = "\t", index = False)

In [ ]:
## Find top Genes for a Cluster
cluster = "AT2 cells"
genes = allMarkers.loc[allMarkers["cluster"] == cluster, "gene"].values

sc.pl.umap(adata, color = genes[0:5], ncols = 5, color_map = gray_red)

In [ ]:
cluster = "B cells"
genes = allMarkers.loc[allMarkers["cluster"] == cluster, "gene"].values

sc.pl.umap(adata, color = genes[0:5], ncols = 5, color_map = gray_red)

### Marker Overview Plots

In [ ]:
sc.pl.rank_genes_groups_dotplot(adata, n_genes = 5)

In [ ]:
sc.pl.rank_genes_groups_matrixplot(adata, n_genes = 5, standard_scale = "var")

In [ ]:
sc.pl.rank_genes_groups_heatmap(adata, n_genes = 5, standard_scale = "var", cmap = "viridis")

In [ ]:
sc.pl.dotplot(adata, var_names = ["Sftpc", "Sftpd", "Tppp3", "Foxj1", "Scgb3a1", "Scgb1a1", "Ager", "Rtkn2", "Krt5"], 
              groupby = "cell_type", cmap = gray_red, dendrogram = True)

In [ ]:
sc.pl.matrixplot(adata, groupby = "cell_type", cmap = "rocket", dendrogram = True, standard_scale = "var",
                 var_names = ["Sftpc", "Sftpd", "Tppp3", "Foxj1", "Scgb3a1", "Scgb1a1","Ager","Rtkn2","Krt5"])

### Re-cluster the object

In [ ]:
sc.tl.leiden(adata, resolution = 2, key_added = "leiden_2")
sc.pl.umap(adata, color = ["leiden_1", "leiden_2"], legend_loc = "on data", legend_fontsize = 15)

There are some subtypes of the endothelial compartment, which can not be easily resolved at this level
- General Capillary Marker: Edn1, Gpihbp1
- Lymphatic endothelial Marker: Prox1, Mmrn1, Maf

In [ ]:
sc.pl.umap(adata, color = ["Edn1", "Gpihbp1", "Prox1", "Mmrn1"], cmap = gray_red, size = 30)

### Generate a subset

In [ ]:
subset = adata[adata.obs.leiden_1.isin(["5"])].copy()
sc.pl.umap(subset, color = ["age", "leiden_1"])

We can highlight the heterogeneity in this subset by re-calculating the Principal components and the neighbourhood graph, to give more weight to genes that distinguish potential endothelial subtypes (rather than those that distinguish endothelial cells from other cell types in the lung)

In [ ]:
## Recalculate PCA
sc.pp.pca(subset, n_comps = 50, use_highly_variable = True)

In [ ]:
## Recalculate k nearest neighbor graph
## Here you can adjust the Parameters n_pcs (how many PCs are being used) n_neighbors (size of a cell’s neighborhood)
sc.pp.neighbors(subset, n_pcs = 10, n_neighbors = 10)
sc.tl.umap(subset)

In [ ]:
sc.pl.umap(subset, color = ["age", "Edn1", "Gpihbp1", "Prox1", "Mmrn1"], ncols = 5, cmap = gray_red)

You can then run a new clustering on this subset as well

In [ ]:
sc.tl.leiden(subset, resolution = 1, key_added = "sub_cluster")
sc.pl.umap(subset, color = ["sub_cluster"], legend_loc = "on data")

In [ ]:
## you can even list more or less annotations in this dictionary
celltype_map = {"8": "Lymphatic EC", "2": "gCap", "3": "gCap", "5": "gCap", "1": "gCap", "0": "gCap",
                "6": "gCap", "10": "gCap", "4": "gCap", "7": "gCap", "9": "gCap"}

subset.obs["cell_type"] = [celltype_map[leiden] if leiden in celltype_map.keys() else leiden
                           for leiden in subset.obs.sub_cluster]

In [ ]:
sc.pl.umap(subset, color = ["cell_type"], legend_loc = "on data", legend_fontsize = 10)

Transfer these more fine-grained labels back to the whole lung object

In [ ]:
## This will only work if both the whole lung and subset column have the same name, here "cell_type"
adata.obs.update(subset.obs.cell_type)
sc.pl.umap(adata, color = ["cell_type"])

### Finally: save object

In [ ]:
subset

In [ ]:
subset.write("Data/endothelial_subset.h5ad")